# Implementing the pre-trained GPT-2 model


Le notebook a pour objectif d'utiliser le modèle pré-entraîné GPT-2, fourni par la bibliothèque [transformers](https://huggingface.co/docs/transformers/index), pour effectuer certaines tâches, sans nécessiter un entraînement préalable spécifique pour celles-ci.

Pour utiliser l'architecture, un tokenizer est utilisé pour encoder les mots, puis le module transformers est utilisé pour générer la sortie, et enfin, le tokenizer est à nouveau utilisé comme décodeur pour décoder la sortie.


In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

Initialisation du modèle et du tokenizer

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model     = GPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)

Test de questions-réponses

In [41]:
test_phrase = "What's the theory of origin of species?"

# We tokenize the input phrase (Encoding)
inputs = tokenizer.encode(test_phrase, return_tensors='pt')

# Use model to generate output text
outputs = model.generate(inputs, max_length=200, do_sample=True, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

# Decode text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

What's the theory of origin of species?

There's a lot of confusion about this, but I think it's important to understand what's going on here. It's not that we don't know what species are there, we know that they exist, and that's what we're trying to do. We want to know how they came to be and what their evolutionary history is. So if you look at the fossil record, you can see that there are a number of different species that have been around for hundreds of thousands of years. There are some that are very close to the end of their life cycle, some of which are quite different from the rest of the species. In fact, there's so much variation in how these species have evolved over time that it can be difficult to make an accurate estimate of how long they've been in existence. But we do have a pretty good idea of what they're capable of doing, so we should be able to say, "Well,


En posant une question du type "Quelle est la capitale de la France ?", nous remarquons que le modèle ne fournit pas la réponse correcte.

In [19]:
test_phrase = "What is the capital of france?"

# We tokenize the input phrase (Encoding)
inputs = tokenizer.encode(test_phrase, return_tensors='pt')

# Use model to generate output text
outputs = model.generate(inputs, max_length=100, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

# Decode text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

What is the capital of france?

France is one of the most popular and well-known brands in the world. It has been around for more than 20 years and has sold over 100 million pairs of shoes. The brand is also known for its high-quality leather, which is made from 100% natural leather. France also offers a wide range of accessories, such as handbags, purses, watches, and more. In addition, the brand has an extensive collection of


Mais si nous fournissons une liste de villes françaises en tant qu'options, il est capable d'identifier et de fournir la réponse correcte.

In [25]:
test_phrase = "What is the capital of france? Annecy, Toulouse, Nantes, Bordeaux, Lyon, Paris, Marseille or Nice?"

# We tokenize the input phrase (Encoding)
inputs = tokenizer.encode(test_phrase, return_tensors='pt')

# Use model to generate output text
outputs = model.generate(inputs, max_length=38, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)

# Decode text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

What is the capital of france? Annecy, Toulouse, Nantes, Bordeaux, Lyon, Paris, Marseille or Nice?

The capital is Paris.


Test d'élaboration d'une histoire à partir d'un contexte

In [28]:
test_phrase = "Once upon a time a small village"

# We tokenize the input phrase (Encoding)
inputs = tokenizer.encode(test_phrase, return_tensors='pt')

# Use model to generate output text
outputs = model.generate(inputs, max_length=100, do_sample=True, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)

# Decode text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

Once upon a time a small village was built on the banks of the river.

The town was named after the king who founded it, and the name was given to the town because it was the place where the people of that time lived. The name of this town has been derived from the fact that it is the only town in the world where there is no water. It is said that the water was used to irrigate the land. In fact, there are no rivers in this world


Test de traduction de mots, avec la stratégie mentionnée dans les articles,
"phrase en anglais = phrase en français ;
phrase en anglais = phrase en français ;
phrase en anglais = phrase en français ;
phrase en anglais = "

Nous constatons qu'il donne la réponse dans la langue attendue (le français), mais la réponse n'est pas correcte.

In [18]:
test_phrase = "The sun is shining brightly = Le soleil brille intensément. I enjoy reading books = J'apprécie lire des livres. Cooking is a creative activity = Cuisiner est une activité créative. When I was your man ="

# We tokenize the input phrase (Encoding)
inputs = tokenizer.encode(test_phrase, return_tensors='pt')

attention_mask = torch.ones(inputs.shape, dtype=torch.long)
# Use model to generate output text
outputs = model.generate(inputs, max_length=68, num_beams=5,attention_mask=attention_mask, no_repeat_ngram_size=2, do_sample = True,top_k=50, top_p=0.95, temperature=0.7)

# Decode text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The sun is shining brightly = Le soleil brille intensément. I enjoy reading books = J'apprécie lire des livres. Cooking is a creative activity = Cuisiner est une activité créative. When I was your man = C'est la même de l'hôtel
